In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from astropy.cosmology import Planck15 as cosmo
from astropy.table import Table
# from archivalCGMtools.loc4 import slime
from scipy.interpolate import LinearNDInterpolator
# from goodies import closest
from mpl_toolkits import mplot3d
from astropy import stats as astats
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pickle
import glob

from pyslime import slime
from pyslime import utils as psu

from numba import jit

import matplotlib
matplotlib.rcParams['font.serif'] = "Times"
matplotlib.rcParams['font.family'] = "serif"

plt.rcParams['mathtext.fontset']='stix'
plt.rcParams['font.size'] = 20


In [1]:
bpDensityFile = '/Users/mwilde/Dropbox/slime-mold/data/final_data/BP_0214_densities_1024_0.bin'
bp_rho_m = np.fromfile(bpDensityFile, dtype=np.float64)
bp_rho_m = np.reshape(bp_rho_m,(1024,1024,1024))

bp_logrhom = np.log10(bp_rho_m)



# random sample of points from the simulation
randx= np.random.randint(0,1023,size=1000)
randy= np.random.randint(0,1023,size=1000)
randz= np.random.randint(0,1023,size=1000)
bp_randrhos = bp_logrhom[randx,randy,randz]
bins = np.arange(-5,3,0.1)
plt.hist(bp_randrhos,bins=bins)
plt.title("BP densities at z=0")
plt.xlabel("density")
print("mean density:",np.mean(10**bp_randrhos))

slime_dirs = glob.glob('/Users/mwilde/Dropbox/slime-mold/data/BP_1024_lin-avg-4x_pow*')
slime_dirs


def get_random_dens_vals(bpslime, size=5000):
    randx = np.random.randint(0, bpslime.griddims[0], size=size)
    randy = np.random.randint(0, bpslime.griddims[1], size=size)
    randz = np.random.randint(0, bpslime.griddims[2], size=size)
    randvals = bpslime.data[randx,randy,randz]
    
    return randvals

bpslime_list = []
bpslime_name = []
slime_density_list = []
for slime_dir in slime_dirs:
    bpslime = slime.Slime.from_dir(slime_dir, axes='xyz')
    bpslime.data = bpslime.data.astype(np.float32)
    bpslime.data = np.log10(bpslime.data)
    bpslime.standardize()
    bpslime_list.append(bpslime)
    
    slime_density_list.append(get_random_dens_vals(bpslime))
    
    name = slime_dir[slime_dir.find('BP_1024'):]
    bpslime_name.append(name)


    ridxs1 = np.random.randint(0,1024,size=10000)
ridxs2 = np.random.randint(0,1024,size=10000)
ridxs3 = np.random.randint(0,1024,size=10000)

size = 1
alpha = 0.25
color = 'tab:blue'

fig, axes = plt.subplots(nrows=len(bpslime_list), sharex=True, figsize=(12,12))

def not_infs(array):
    return array[~np.isinf(array)]


axes[0].scatter(bp_logrhom[ridxs1, ridxs2, ridxs3], 
                bpslime_list[1].data[ridxs1, ridxs2, ridxs3],
                s=size, 
                alpha=alpha,
                c=color)
axes[0].text(0.1, 0.75, s=r'exp = 2.5', transform=axes[0].transAxes)


axes[1].scatter(bp_logrhom[ridxs1, ridxs2, ridxs3], bpslime_list[2].data[ridxs1, ridxs2, ridxs3],
                s=size,
                alpha=alpha)
axes[1].text(0.1, 0.75, s=r'exp = 3.0', transform=axes[1].transAxes)
axes[1].set_ylabel(r'log $\rho_m / \langle \rho \rangle$')

axes[2].scatter(bp_logrhom[ridxs1, ridxs2, ridxs3], bpslime_list[0].data[ridxs1, ridxs2, ridxs3],
                s=size,
                alpha=alpha)
axes[2].text(0.1, 0.75, s=r'exp = 3.5', transform=axes[2].transAxes)

axes[2].set_xlabel('log MCPM density');
plt.subplots_adjust(hspace=0)

# plt.savefig('bp_sm_z0_power_law_comparison.pdf')


ridxs1 = np.random.randint(0,1024,size=10000)
ridxs2 = np.random.randint(0,1024,size=10000)
ridxs3 = np.random.randint(0,1024,size=10000)

size = 1
alpha = 0.25
color = 'tab:blue'

def not_infs(array1, array2):
    return array1[~np.isinf(array1) & ~np.isinf(array2)], array2[~np.isinf(array1) & ~np.isinf(array2)]



bp_clean25, s_clean25 = not_infs(bp_logrhom[ridxs1, ridxs2, ridxs3], bpslime_list[1].data[ridxs1, ridxs2, ridxs3])
bp_clean30, s_clean30 = not_infs(bp_logrhom[ridxs1, ridxs2, ridxs3], bpslime_list[2].data[ridxs1, ridxs2, ridxs3])
bp_clean35, s_clean35 = not_infs(bp_logrhom[ridxs1, ridxs2, ridxs3], bpslime_list[0].data[ridxs1, ridxs2, ridxs3])

NameError: name 'np' is not defined

In [ ]:

fig, axes = plt.subplots(nrows=len(bpslime_list), sharex=False, figsize=(10, 12))

axes[0].hist2d(bp_clean25, 
                s_clean25, 
               bins=(90, 40),
               cmap='Greens')
axes[0].text(0.1, 0.75, s=r'exp = 2.5', transform=axes[0].transAxes)
axes[0].set_xlim(-1.6, 1.1)
axes[0].set_xticks([])



axes[1].hist2d(bp_clean30, s_clean30, bins=(90, 40), cmap='Greens')
axes[1].text(0.1, 0.75, s=r'exp = 3.0', transform=axes[1].transAxes)
axes[1].set_ylabel(r'log $\rho_m / \langle \rho \rangle$')
axes[1].set_xlim(-1.6, 1.1)
axes[1].set_xticks([])

axes[2].hist2d(bp_clean35, s_clean35, bins=(90, 40), cmap='Greens')
axes[2].text(0.1, 0.75, s=r'exp = 3.5', transform=axes[2].transAxes)
axes[2].set_xlim(-1.6, 1.1)

axes[2].set_xlabel(r'$\rm log~MCPM~density$ ');
plt.subplots_adjust(hspace=0.05)

plt.savefig('Figures/bp_sm_z0_power_law_comparison.pdf')